In [1]:
import os
import random
import math
from sklearn.metrics import mean_squared_error
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sn
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pmdarima as pm 
from sklearn.model_selection import train_test_split
from pmdarima.pipeline import Pipeline

In [2]:
os.getcwd()

'C:\\Users\\niles\\project\\project_work\\notebooks'

In [3]:
# Read data:
#os.chdir("..")
df = pd.read_csv("data.csv")

In [4]:
df.head()

,Time,air_pressure[mmHg],air_temperature[degree celcius],relative_humidity[%],wind_speed[M/S],solar_irridiation[W/m²],total_cloud_cover[from ten],electricity_demand_values[kw],heat_demand_values[kw]
0,2010-12-01 00:00:00,729.7,25.0,85.0,5.0,0,no clouds,289.567557,432.940036
1,2010-12-01 01:00:00,729.4,27.8,77.0,7.0,0,no clouds,260.168520,473.935901
2,2010-12-01 02:00:00,728.9,33.3,62.0,7.0,0,2/10–3/10.,247.273585,483.278761
3,2010-12-01 03:00:00,731.6,32.2,62.0,2.0,0,5/10.,257.955878,545.921252
4,2010-12-01 04:00:00,732.6,22.8,96.0,3.0,0,2/10–3/10.,258.255081,550.526112


In [5]:
len(df)

70080

In [6]:
df = df.dropna()

In [7]:
len(df)

68792

In [8]:
# Rename Columns:
col_mapping = {"Time": "date",
            "air_pressure[mmHg]": "air_pressure",
            "air_temperature[degree celcius]": "air_temperature",
            "relative_humidity[%]": "relative_humidity",
            "wind_speed[M/S]": "wind_speed",
            "solar_irridiation[W/m²]": "solar_irridiation",
            "total_cloud_cover[from ten]": "total_cloud_cover",
            "electricity_demand_values[kw]": "electricity",
            "heat_demand_values[kw]": "heat"}
df = df.rename(columns = col_mapping)

In [9]:
# Change ts to datetime and set as index:
df["date"] = pd.to_datetime(df["date"],format='%Y-%m-%d')
df.head()

,date,air_pressure,air_temperature,relative_humidity,wind_speed,solar_irridiation,total_cloud_cover,electricity,heat
0,2010-12-01 00:00:00,729.7,25.0,85.0,5.0,0,no clouds,289.567557,432.940036
1,2010-12-01 01:00:00,729.4,27.8,77.0,7.0,0,no clouds,260.168520,473.935901
2,2010-12-01 02:00:00,728.9,33.3,62.0,7.0,0,2/10–3/10.,247.273585,483.278761
3,2010-12-01 03:00:00,731.6,32.2,62.0,2.0,0,5/10.,257.955878,545.921252
4,2010-12-01 04:00:00,732.6,22.8,96.0,3.0,0,2/10–3/10.,258.255081,550.526112


In [10]:
len(df)

68792

In [11]:
df.head()

,date,air_pressure,air_temperature,relative_humidity,wind_speed,solar_irridiation,total_cloud_cover,electricity,heat
0,2010-12-01 00:00:00,729.7,25.0,85.0,5.0,0,no clouds,289.567557,432.940036
1,2010-12-01 01:00:00,729.4,27.8,77.0,7.0,0,no clouds,260.168520,473.935901
2,2010-12-01 02:00:00,728.9,33.3,62.0,7.0,0,2/10–3/10.,247.273585,483.278761
3,2010-12-01 03:00:00,731.6,32.2,62.0,2.0,0,5/10.,257.955878,545.921252
4,2010-12-01 04:00:00,732.6,22.8,96.0,3.0,0,2/10–3/10.,258.255081,550.526112


In [12]:
df1 = df[["date",'electricity']]

In [13]:
df1 = df1.set_index("date")
df1.head()

,electricity
date,
2010-12-01 00:00:00,289.567557
2010-12-01 01:00:00,260.168520
2010-12-01 02:00:00,247.273585
2010-12-01 03:00:00,257.955878
2010-12-01 04:00:00,258.255081


In [14]:
x = df1['electricity'].values.reshape(-1, 1) #returns a numpy array
min_max_scaler = StandardScaler()
x_scaled = min_max_scaler.fit_transform(x)
df1['electricity'] = x_scaled

In [15]:
df1.head()

,electricity
date,
2010-12-01 00:00:00,-0.437728
2010-12-01 01:00:00,-0.560519
2010-12-01 02:00:00,-0.614377
2010-12-01 03:00:00,-0.569760
2010-12-01 04:00:00,-0.568511


In [16]:
davg_df = df1.groupby(pd.Grouper(freq='D')).mean()
davg_df.head()

,electricity
date,
2010-12-01,-0.159240
2010-12-02,-0.270442
2010-12-03,-0.122860
2010-12-04,0.095403
2010-12-05,0.213953


In [17]:
len(davg_df)

2920

In [18]:
dftrain, dftest = train_test_split(davg_df, test_size=0.01*7)

In [19]:
len(dftest)

205

In [20]:
dftrain, dfvali = train_test_split(dftrain, test_size=0.2)

In [21]:
len(dftrain)

2172

In [22]:
len(dfvali)

543

In [23]:
# best
random.seed(0)

In [24]:
'''
pipeline = Pipeline([
    ("model", pm.AutoARIMA(test='adf',
                           start_p=3, start_q=3, max_p=5, max_q=5, start_P=0, start_Q=0, max_P=5, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='aic', trace=True, d=1, D=1, error_action='warn', 
                           suppress_warnings=True, random_state=20, n_fits=100))
])
'''
pipeline = Pipeline([
    ("model",pm.AutoARIMA(test='adf',
                           max_p=3, max_q=3, m=7,
                           start_P=1,start_Q=1, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=False))
])
pipeline.fit(dftrain)

 ARIMA(0,1,0)(0,1,0)[7]             : AIC=6666.674, Time=0.05 sec
 ARIMA(0,1,0)(0,1,1)[7]             : AIC=inf, Time=1.07 sec
 ARIMA(0,1,0)(0,1,2)[7]             : AIC=inf, Time=1.76 sec
 ARIMA(0,1,0)(1,1,0)[7]             : AIC=6080.015, Time=0.13 sec
 ARIMA(0,1,0)(1,1,1)[7]             : AIC=inf, Time=0.74 sec
 ARIMA(0,1,0)(1,1,2)[7]             : AIC=inf, Time=2.96 sec
 ARIMA(0,1,0)(2,1,0)[7]             : AIC=5855.773, Time=0.20 sec
 ARIMA(0,1,0)(2,1,1)[7]             : AIC=inf, Time=2.64 sec
 ARIMA(0,1,0)(2,1,2)[7]             : AIC=inf, Time=2.36 sec
 ARIMA(0,1,1)(0,1,0)[7]             : AIC=inf, Time=0.36 sec
 ARIMA(0,1,1)(0,1,1)[7]             : AIC=inf, Time=1.08 sec
 ARIMA(0,1,1)(0,1,2)[7]             : AIC=inf, Time=2.62 sec
 ARIMA(0,1,1)(1,1,0)[7]             : AIC=inf, Time=1.16 sec
 ARIMA(0,1,1)(1,1,1)[7]             : AIC=inf, Time=1.56 sec
 ARIMA(0,1,1)(1,1,2)[7]             : AIC=inf, Time=3.44 sec
 ARIMA(0,1,1)(2,1,0)[7]             : AIC=inf, Time=2.45 sec
 ARIMA(0,

Pipeline(steps=[('model',
                 AutoARIMA(D=1, d=1, error_action='ignore', m=7, max_p=3,
                           max_q=3, stepwise=False, test='adf', trace=True))])

In [47]:
##### univariate arima model predict electricity_demand_values  
# best
#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=5, start_P=0, start_Q=0, max_P=5, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='aic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, random_state=20, n_fits=100)

#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=6, start_P=0, start_Q=4, max_P=6, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='aic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, random_state=20, n_fits=100)

#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=6, start_P=0, start_Q=4, max_P=6, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='bic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, n_fits=100)

#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=6, start_P=1, start_Q=4, max_P=6, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='bic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, n_fits=100)


In [27]:
Sample_predictions= pipeline.predict_in_sample(exogenous=None, return_conf_int=False, alpha=0.05, inverse_transform=True)
Actuals = pd.Series.to_numpy(dftrain['electricity'])
Residuals = Actuals-Sample_predictions

In [28]:
Residuals

array([-0.72214325,  0.15951395,  0.3021969 , ..., -0.48796626,
       -0.96189234,  0.95798674])

In [30]:
forecast, conf = pipeline.predict(n_periods=543, return_conf_int=True)
forecast

array([ 2.73597718e-01, -1.76508738e-01, -1.52979286e-02,  5.37344587e-01,
        1.34216454e-01,  2.59622188e-01, -2.20252425e-01,  3.96087393e-01,
       -3.89417864e-02,  9.09559587e-02,  1.29382026e-01,  1.29193361e-01,
        2.10864645e-01, -9.08171708e-02,  2.50177622e-01, -1.78481496e-02,
        2.92689432e-01,  6.47924446e-03, -7.50955944e-03,  3.24394347e-02,
        2.24035926e-02,  2.47392834e-01, -1.33280027e-01,  7.08722187e-02,
        1.60630611e-01,  2.50343182e-02,  1.06050946e-01, -1.49822247e-01,
        2.39185978e-01, -1.22923752e-01,  9.13319818e-02,  4.37451589e-02,
       -8.94216877e-03,  5.91003868e-02, -1.32677353e-01,  1.88114237e-01,
       -1.49479369e-01,  9.27205994e-02,  1.19364369e-02, -5.48360956e-02,
        8.27195903e-03, -1.45214632e-01,  1.66447221e-01, -1.92972670e-01,
        2.79301450e-02,  1.30485369e-02, -7.14953660e-02, -1.00184166e-03,
       -1.99948946e-01,  1.39698935e-01, -2.13584195e-01,  1.20036262e-02,
       -3.46323640e-02, -

In [31]:
Actuals = pd.Series.to_numpy(dfvali['electricity'])
Actuals

array([-6.29973049e-01,  1.52433125e+00,  4.92967106e-01, -1.03015742e-01,
       -4.90715499e-01, -7.15777783e-01, -3.26898045e-01,  2.63512793e-02,
       -5.40543066e-01, -3.24636149e-01, -6.70306657e-01,  3.31667274e-01,
       -3.18118828e-01,  2.30737935e-01, -4.61319053e-01,  9.80246359e-01,
       -5.15376110e-01, -3.23612689e-01, -4.90112946e-01, -4.92015432e-01,
       -3.44841343e-01, -6.45460174e-01,  2.14841313e-01,  7.00844280e-01,
        3.34001574e-01,  1.61619834e-01,  1.72123924e+00, -2.02628762e-01,
        3.30282630e-01,  2.55347277e-01, -5.11766223e-01, -1.15806972e-01,
        2.75287148e-01, -9.51951916e-02, -1.84635194e-01,  1.25004754e-01,
        2.64754720e-02, -5.57295130e-01, -4.43367251e-01, -2.48306060e-01,
       -3.64828778e-01,  1.74669837e-01, -1.93557352e-01, -1.40877504e-01,
        8.02558882e-02,  3.82603730e-01,  3.64958581e-01,  4.67841879e-01,
       -3.41000555e-01, -5.47321431e-01, -2.12736520e-01, -6.75369378e-01,
        2.14740891e-01, -

In [32]:
mseVali = mean_squared_error(Actuals, forecast)
mseVali

1.7235935235383337

In [33]:
forecasttrain, _ = pipeline.predict(n_periods=2172, return_conf_int=True)
forecasttrain

array([ 0.27359772, -0.17650874, -0.01529793, ..., -9.00291343,
       -8.67553581, -9.02824808])

In [34]:
Actualstrain = pd.Series.to_numpy(dftrain['electricity'])
Actualstrain

array([-0.72214325, -0.56262906, -0.2604323 , ..., -0.1212007 ,
       -0.21138998,  0.56397838])

In [35]:
msetrain = mean_squared_error(Actualstrain, forecasttrain)
msetrain

26.128617384700497

In [36]:
forecasttest, conf = pipeline.predict(n_periods=205, return_conf_int=True)
forecasttest

array([ 0.27359772, -0.17650874, -0.01529793,  0.53734459,  0.13421645,
        0.25962219, -0.22025243,  0.39608739, -0.03894179,  0.09095596,
        0.12938203,  0.12919336,  0.21086465, -0.09081717,  0.25017762,
       -0.01784815,  0.29268943,  0.00647924, -0.00750956,  0.03243943,
        0.02240359,  0.24739283, -0.13328003,  0.07087222,  0.16063061,
        0.02503432,  0.10605095, -0.14982225,  0.23918598, -0.12292375,
        0.09133198,  0.04374516, -0.00894217,  0.05910039, -0.13267735,
        0.18811424, -0.14947937,  0.0927206 ,  0.01193644, -0.0548361 ,
        0.00827196, -0.14521463,  0.16644722, -0.19297267,  0.02793015,
        0.01304854, -0.07149537, -0.00100184, -0.19994895,  0.13969893,
       -0.21358419,  0.01200363, -0.03463236, -0.10315094, -0.03582271,
       -0.21785081,  0.10670313, -0.24346815, -0.01400348, -0.06150517,
       -0.13460364, -0.06762611, -0.24584126,  0.07938301, -0.27478332,
       -0.04933686, -0.08599405, -0.16132948, -0.09310041, -0.27

In [37]:
Actualstest = pd.Series.to_numpy(dftest['electricity'])
Actualstest

array([ 1.17559538, -0.33244801,  0.83141088,  0.24584192, -0.32697123,
       -0.50282276, -0.09413299,  0.24302166, -0.14782673, -0.49030966,
        0.21191174, -0.45970528, -0.0080224 ,  0.26035081,  1.10418766,
        0.09843596, -0.26582991, -0.22159887,  0.45849001, -0.41607271,
       -0.55367848,  0.10064829,  0.36586604, -0.53729269, -0.39815368,
       -0.35082935, -0.1555076 , -0.64066743,  0.55783498,  0.58634523,
       -0.71601611, -0.47086407, -0.26380545, -0.34569   , -0.32195181,
        0.37808895,  0.44328094,  0.085156  , -0.661428  ,  1.06887189,
       -0.01662352, -0.0690164 , -0.61743582, -0.13630956, -0.72214325,
       -0.18255226, -0.36354632, -0.23492332,  0.65426748,  0.06675377,
        1.44152014, -0.72214325,  0.04300189,  0.56060975,  0.45577163,
       -0.45237475,  0.06442793,  0.19011825, -0.64823753,  0.02206108,
       -0.34354355,  0.03462173, -0.52537735,  0.36020242,  0.36575572,
       -0.18582463, -0.62016832, -0.16867445, -0.30868163,  0.41

In [38]:
msetest = mean_squared_error(Actualstest, forecasttest)
msetest

0.44697815595926715

In [39]:
rmse = math.sqrt(msetest)
print('RMSE: %f' % rmse)

RMSE: 0.668564


In [52]:
mse = mean_squared_error(Actuals, forecast)
mse

8.534984766097647

In [53]:
rmse = math.sqrt(mse)
print('RMSE: %f' % rmse)

RMSE: 2.921470
